# Homework - Week 1 Day 2: Resumidor de Websites com Ollama

## Objetivo

Usar um modelo open-source rodando localmente via **Ollama** para resumir o conteudo de um website em portugues brasileiro.

## Como funciona

1. **Scraping do website** - A funcao `fetch_website_contents` (do modulo `scraper.py`) faz uma requisicao HTTP ao site, extrai o texto do `<body>` removendo scripts, estilos e imagens, e retorna o titulo + conteudo truncado em 2.000 caracteres.

2. **Ollama como backend local** - O Ollama expoe um endpoint compativel com a API da OpenAI em `http://localhost:11434/v1`. Isso permite usar o pacote `openai` do Python como client, apenas apontando o `base_url` para o servidor local. Nenhum dado sai da sua maquina.

3. **Prompts em PT-BR** - O system prompt instrui o modelo a responder em portugues brasileiro, com um tom sarcastico e humoristico. O user prompt envia o conteudo extraido do site e pede um resumo.

4. **Exibicao em Markdown** - A resposta do modelo e renderizada diretamente como Markdown no notebook.

## Pre-requisitos

- Ollama instalado e rodando (`ollama serve`)
- Modelo baixado (ex: `ollama pull phi3` ou `ollama pull llama3.2`)
- Pacotes Python: `openai`, `beautifulsoup4`, `requests`

In [10]:
import sys
sys.path.append('..')

from scraper import fetch_website_contents
from IPython.display import Markdown, display
from openai import OpenAI

OLLAMA_BASE_URL = "http://localhost:11434/v1"
MODEL = "phi4-mini:latest"

ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key="ollama")
print(f"Cliente Ollama configurado com modelo: {MODEL}")

Cliente Ollama configurado com modelo: phi4-mini:latest


In [11]:
import subprocess

# Verifica se o modelo ja esta instalado, senao faz o pull
installed = subprocess.run(["ollama", "list"], capture_output=True, text=True)
if MODEL not in installed.stdout:
    print(f"Modelo {MODEL} nao encontrado. Baixando...")
    subprocess.run(["ollama", "pull", MODEL])
    print(f"Modelo {MODEL} instalado com sucesso!")
else:
    print(f"Modelo {MODEL} ja esta instalado.")

Modelo phi4-mini:latest ja esta instalado.


In [12]:
system_prompt = """
Voce e um assistente sarcastico que analisa o conteudo de um website
e fornece um resumo curto, sarcastico e bem-humorado, ignorando texto de navegacao.
Responda sempre em portugues brasileiro.
Responda em markdown. Nao coloque o markdown dentro de um bloco de codigo - responda direto com o markdown.
"""

user_prompt_prefix = """
Aqui esta o conteudo de um website.
Faca um resumo curto deste site.
Se houver noticias ou anuncios, resuma-os tambem.

"""


def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_prefix + website}
    ]


def summarize(url):
    website = fetch_website_contents(url)
    response = ollama.chat.completions.create(
        model=MODEL,
        messages=messages_for(website)
    )
    return response.choices[0].message.content


def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))


display_summary("https://anthropic.com")

### Resumo Curto e Sarcástico:

**🌌 Anthropic 🚀:** Onde a privacidade não descansa em salada.

- **Noticias Nerds (Febrão de Feb!):**
    - 🎓 *Opus 4.6*: Não se preocupem com as tarefas banais, temos agora um super-herói do código para essas infernais: Claude Opis dos Coding Machines.
    
**Nota Sarcástica:** Vamos esperar até que possamos mandar o AIA a fazer nossas pipocas e pizzas! 🏠👨‍💻🍽️

*Sem contar as "features" fancy, mas estamos na braçada de um futuro no qual as IA podem se lembrar do seu nome (embora preferisse não)? #FutureIsWhatWeMakeOf*"